In [1]:
import os
import fitz
from paddleocr import PaddleOCR
import torch

In [2]:
filename = os.path.join("pdf", "联通手机营业厅使用手册.pdf")
doc = fitz.open(filename)

In [4]:
page_count = doc.page_count
img_count = 0
img_dir = "img"
os.makedirs(img_dir, exist_ok=True)
for page_idx in range(page_count):
    # page = doc.load_page(page_idx)
    # text = page.get_text()
    # print(text)
    img_list = doc.get_page_images(page_idx)
    print(f"page[{page_idx}]: image num={len(img_list)}")
    for img in img_list:
        pix = fitz.Pixmap(doc, img[0])
        if pix.n - pix.alpha >= 4:
            pix = fitz.Pixmap(fitz.csRGB, pix)
        pix.save(os.path.join(img_dir, f"{img_count}.png"))
        img_count += 1

page[0]: image num=0
page[1]: image num=0
page[2]: image num=1
page[3]: image num=2
page[4]: image num=2
page[5]: image num=2
page[6]: image num=2
page[7]: image num=2
page[8]: image num=2
page[9]: image num=2
page[10]: image num=2
page[11]: image num=2
page[12]: image num=3
page[13]: image num=0


In [ ]:
def pdf_ocr_txt(filepath, dir_path="tmp_files"):
    full_dir_path = os.path.join(os.path.dirname(filepath), dir_path)
    if not os.path.exists(full_dir_path):
        os.makedirs(full_dir_path)
    ocr = PaddleOCR(use_angle_cls=True, lang="ch", use_gpu=torch.cuda.is_available(), show_log=False)
    doc = fitz.open(filepath)
    txt_file_path = os.path.join(full_dir_path, f"{os.path.split(filepath)[-1]}.txt")
    img_name = os.path.join(full_dir_path, 'tmp.png')
    with open(txt_file_path, 'w', encoding='utf-8') as fout:
        for i in range(doc.page_count):
            page = doc[i]
            text = page.get_text("")
            fout.write(text)
            fout.write("\n")

            img_list = page.get_images()
            for img in img_list:
                pix = fitz.Pixmap(doc, img[0])
                if pix.n - pix.alpha >= 4:
                    pix = fitz.Pixmap(fitz.csRGB, pix)
                pix.save(img_name)

                result = ocr.ocr(img_name)
                ocr_result = [i[1][0] for line in result for i in line]
                fout.write("\n".join(ocr_result))
    if os.path.exists(img_name):
        os.remove(img_name)
    return txt_file_path

# Split pdf with outline

In [5]:
import fitz
import os
from pprint import pprint

In [4]:
filename = "pdf/《中国联通客服知识库》3G业务客服文档.pdf"
doc = fitz.open(filename)

In [32]:
from doc_splitter import match_and_split_heading, postprocess

def check_heading_match(heading_text, paragraphs, start_idx):
    heading_text = heading_text.replace(" ", "")
    paragraph = paragraphs[start_idx].replace(" ", "")
    if heading_text == paragraph:
        return True, start_idx
    if heading_text.startswith(paragraph) and start_idx+1 < len(paragraphs):
        heading_text = heading_text[len(paragraph):]
        return check_heading_match(heading_text, paragraphs, start_idx+1)
    return False, start_idx

def split_pdf_by_heading(filename):
    doc = fitz.open(filename)
    tocs = doc.get_toc()
    paragraphs = "".join(page.get_text() for page in doc).split("\n")
    doc_tree = {}
    heading_stack = []
    outline_idx, para_idx = 0, 0

    while para_idx < len(paragraphs):
        paragraph = paragraphs[para_idx]
        if outline_idx < len(tocs):
            outline_level, outline_text = tocs[outline_idx][:2]
            is_match, start_idx = check_heading_match(outline_text, paragraphs, para_idx)
        else:
            is_match = False
        if is_match:
            outline_idx += 1
            para_idx = start_idx
            level = int(outline_level)
            heading_stack = heading_stack[:level-1]
            heading_rank, heading_text = match_and_split_heading(outline_text)
            if heading_rank is None:
                print("Parse Heading Error: ", paragraph)
            heading_stack.append((level, heading_text))
        else:
            if len(heading_stack) > 0 and len(paragraph) > 0:
                key = "#".join([x[1] for x in heading_stack])
                if key not in doc_tree:
                    doc_tree[key] = ""
                doc_tree[key] += postprocess(paragraph)
        para_idx += 1
    return doc_tree


doc_tree = split_pdf_by_heading(filename)
pprint(doc_tree)


Parse Heading Error:  2.联通 3G 优势
Parse Heading Error:  11.业务营销策略
Parse Heading Error:  2.包天、包周及包月的 3G 增值业务计费原那么
Parse Heading Error:  8.促销活动
Parse Heading Error:  3.协议补贴 3G 客户开户办理手续
Parse Heading Error:  4.客户变更 3G 差不多套餐/可选包套餐方式
Parse Heading Error:  5.后付费 3G 客户停机保号/开机办理方法
Parse Heading Error:  8.滚动预存话费信用额度调整
Parse Heading Error:  22.向 3G 客户提供的异地业务及服务
Parse Heading Error:  7．3G 国际漫游国家与网络频率列表
Parse Heading Error:  1.标准资费
Parse Heading Error:  2.套餐资费
Parse Heading Error:  3.资费说明
Parse Heading Error:  1.入网
Parse Heading Error:  2.后付费产品停/开机
Parse Heading Error:  3.预付费产品挂失/解挂
Parse Heading Error:  4.预付费产品二次入网
Parse Heading Error:  6.预付费上网卡服务范畴〔营业厅、客服热线办理随 BSS 系统的建设逐步过渡到 BSS 系统支撑〕
Parse Heading Error:  服务内容
Parse Heading Error:  服务渠道
Parse Heading Error:  咨询
Parse Heading Error:  网上营业厅
Parse Heading Error:  使用地营业厅
Parse Heading Error:  使用地客服
Parse Heading Error:  交费
Parse Heading Error:  网上营业厅
Parse Heading Error:  使用地营业厅〔举荐或引导
Parse Heading Error:  一卡充
Parse Heading Error:  查
Parse Heading E

In [20]:
paragraphs = "".join(page.get_text() for page in doc).split("\n")
pprint(paragraphs)

['《中国联通客服知识库》3G 业务客服文档',
 '——3G 业务客服文档',
 '第一部分',
 '3G 基础知识',
 '一、业务介绍',
 '1.3G 网络的定义',
 '3G 是指第三代移动通信技术，是将无线通信与互联网等多媒体通信结合的新一代移动通信系统，能',
 '够提供包括可视',
 '、无线上网，手机上网、手机电视、手机音乐等多种信息服务。中国联通采纳的是',
 'WCDMA 通信标准。',
 '2.联通 3G 优势',
 '\uf0d8 与 2G 网络相比：',
 '\uf0b2',
 '速度更快、选择更个性化、网络覆盖更宽广、业务更丰富。',
 '\uf0d8 与其它 3G 标准相比：',
 '\uf0b2',
 '全球开通网络最多。截至 2020 年底，全球有 264 个 WCDMA 网络，占全球 3G 商用网络的',
 '71.3%。',
 '\uf0b2',
 '开通国家最广。截至 2020 年底，全球 115 个国家具有 WCDMA 网络覆盖。',
 '\uf0b2',
 '全球市场份额最高。截至 2020 年底，全球共有 WCDMA/HSPA/GSM 用户 3.54 亿，全球市场',
 '占有率 89.5%。',
 '\uf0b2',
 '终端款式最丰富。截至 2020 年底，支持 WCDMA 商用终端的款式数量超过 2000 款。',
 '\uf0b2',
 '商用时刻最长，技术成熟、可演进性最好。全球第一个 3G 商用网络确实是采纳 WCDMA 制',
 '式。',
 '3.3G 用户的定义',
 '开通使用中国联通 3G 标准资费、3G 差不多套餐、无线上网卡标准资费或套餐的用户。',
 '4.3G 后付费用户定义',
 '3G 后付费用户须签约和登记用户资料；先使用，后付费，按月缴费，享受客户俱乐部、积分等客户',
 '服务。可对后付费用户进行信用额度操纵 。',
 '5.3G 标准产品〔预付费〕定义',
 '3G 标准产品〔预付费〕是实时扣费、先付费后使用的 3G 产品。',
 '6.3G 标准产品〔预付费〕用户定义',
 '使用 3G 标准产品〔预付费〕的用户，无须登记客户资料，先付费后使用，实时扣费；以自助服务',
 '为主。',
 '7.3G 重点业务',
 '\uf0d83G 重点业务包括：无线上网卡、手机电视、手机

In [29]:

tocs = doc.get_toc()
# pprint(tocs)
first_outline_level, first_outline_text = tocs[0][:2]
is_match, start_idx = check_heading_match(first_outline_text, paragraphs, 2)
print(is_match, start_idx)

Check Heading Match:  第一部分3G基础知识 第一部分
Startswith
Remaining Heading:  第一部分
Check Heading Match:  第一部分 3G基础知识
False 3


# Split problem set book in pdf format

In [11]:
import fitz
import os
from pprint import pprint
from doc_splitter import match_and_split_heading, postprocess
import json

In [17]:
def check_heading_match(heading_text, paragraphs, start_idx, if_first=True):
    heading_text = heading_text.replace(" ", "")
    paragraph = paragraphs[start_idx].replace(" ", "")
    if paragraph == heading_text:
        res = "Equal" if if_first else "Contain"
        print(res, heading_text, paragraph)
        return res, start_idx
    if paragraph.startswith(heading_text):
        print("Contain", heading_text, paragraph)
        return "Contain", start_idx
    if heading_text.startswith(paragraph) and start_idx+1 < len(paragraphs):
        heading_text = heading_text[len(paragraph):]
        return check_heading_match(heading_text, paragraphs, start_idx+1, if_first=False)
    print("Fail", heading_text, paragraph)
    return "Fail", start_idx

def split_pdf_by_heading(filename):
    doc = fitz.open(filename)
    tocs = doc.get_toc()
    paragraphs = "".join(page.get_text() for page in doc).replace("\xad", "-").replace("\xa0", "").split("\n")
    doc_tree = {}
    heading_stack = []
    outline_idx, para_idx = 0, 0

    while para_idx < len(paragraphs):
        paragraph = paragraphs[para_idx]
        if outline_idx < len(tocs):
            outline_level, outline_text = tocs[outline_idx][:2]
            is_match, start_idx = check_heading_match(outline_text, paragraphs, para_idx)
        else:
            is_match = "Fail"
        if is_match in ["Equal", "Contain"]:
            outline_idx += 1
            para_idx = start_idx
            level = int(outline_level)
            heading_stack = heading_stack[:level-1]
            heading_rank, heading_text = match_and_split_heading(paragraph)
            if heading_rank is None:
                print("Parse Heading Error: ", paragraph)
                heading_stack.append((level, outline_text))
                key = "#".join([x[1] for x in heading_stack])
                if key not in doc_tree:
                    doc_tree[key] = ""
                doc_tree[key] += postprocess(heading_text) + "\n"
            elif is_match == "Equal":
                heading_stack.append((level, heading_text))
            elif is_match == "Contain":
                heading_stack.append((level, heading_rank))
                key = "#".join([x[1] for x in heading_stack])
                if key not in doc_tree:
                    doc_tree[key] = ""
                doc_tree[key] += postprocess(heading_text) + "\n"
        else:
            if len(heading_stack) > 0 and len(paragraph) > 0:
                key = "#".join([x[1] for x in heading_stack])
                if key not in doc_tree:
                    doc_tree[key] = ""
                doc_tree[key] += postprocess(paragraph) + "\n"
        para_idx += 1
    return doc_tree

def split_qa(doc_tree, output_filename):
    with open(output_filename, "w", encoding="utf-8") as fout:
        for key, value in doc_tree.items():
            value_list = value.split("\n")
            for idx in range(len(value_list)):
                found = False
                if value_list[idx].strip().startswith("答：") or value_list[idx].strip().startswith("解答："):
                    fout.write(json.dumps({
                            "instruction": key,
                            "question": "".join(value_list[:idx]),
                            "answer": "".join(value_list[idx:])
                        }, ensure_ascii=False) + "\n")
                    found = True
                    break
            if not found:
                fout.write(json.dumps({
                        "instruction": key,
                        "question": "".join(value_list),
                        "answer": ""
                    }, ensure_ascii=False) + "\n")

In [19]:
filename = "problems/Unlock-《计算机网络教程》第五版--谢希仁编-电子工业出版社课件和课后答案.pdf"
doc_tree = split_pdf_by_heading(filename)
pprint(doc_tree)
print(len(doc_tree))
# doc = fitz.open(filename)
# paragraphs = "".join(page.get_text() for page in doc).replace("\xad", "-").replace("\xa0", "").split("\n")
# pprint(paragraphs)
# tocs = doc.get_toc()
# pprint(tocs)
output_filename = filename + ".jsonl"
split_qa(doc_tree, output_filename)

Fail 习题参考答案 爱答案习题答案课件资源网www.aidaan.cn
Fail 习题参考答案 韶关学院信息工程学院骆耀祖整理
Fail 习题参考答案 谢希仁《计算机网络教程》
Equal 习题参考答案 习题参考答案
Parse Heading Error:  习题参考答案 
Equal 第一章概述 第一章概述
Fail 1-01 传播时延＝信道长度/电磁波在信道上的传播速度
Fail 1-01 发送时延＝数据块长度/信道带宽
Fail 1-01 总时延＝传播时延＋发送时延＋排队时延
Contain 1-01 1-01计算机网络的发展可划分为几个阶段？每个阶段各有何特点？
Fail 1-02 答：计算机网络的发展可分为以下四个阶段。
Fail 1-02 （1）面向终端的计算机通信网：其特点是计算机是网络的中心和控制者，终端围绕中心
Fail 1-02 计算机分布在各处，呈分层星型结构，各终端通过通信线路共享主机的硬件和软件资源，计
Fail 1-02 算机的主要任务还是进行批处理，在20世纪60年代出现分时系统后，则具有交互式处理和
Fail 1-02 成批处理能力。
Fail 1-02 （2）分组交换网：分组交换网由通信子网和资源子网组成，以通信子网为中心，不仅共
Fail 1-02 享通信子网的资源，还可共享资源子网的硬件和软件资源。网络的共享采用排队方式，即由
Fail 1-02 结点的分组交换机负责分组的存储转发和路由选择，给两个进行通信的用户断续（或动态）
Fail 1-02 分配传输带宽，这样就可以大大提高通信线路的利用率，非常适合突发式的计算机数据。
Fail 1-02 （3）形成计算机网络体系结构：为了使不同体系结构的计算机网络都能互联，国际标准
Fail 1-02 化组织ISO提出了一个能使各种计算机在世界范围内互联成网的标准框架—开放系统互连基
Fail 1-02 本参考模型OSI.。这样，只要遵循OSI标准，一个系统就可以和位于世界上任何地方的、也
Fail 1-02 遵循同一标准的其他任何系统进行通信。
Fail 1-02 （4）高速计算机网络：其特点是采用高速网络技术，综合业务数字网的实现，多媒体和
Fail 1-02 智能型网络的兴起。
Contain 1-02 1-02试简述分组交换的要点。
Fail 1-03 答：分组交换实质上是